# Model Comparison Experiment

This notebook compares **efficiency metrics only** across different Gemini LLM models:
- **gemini-2.0-flash** (Balanced - baseline)
- **gemini-2.5-pro** (High quality, slower)
- **gemini-2.0-flash-lite** (Optimized for speed)

**Evaluation Scope**: Pure efficiency comparison (no backtesting)

**Metrics Tracked**:
- Time/Round
- Time/Agent
- Avg Rounds
- Total Time
- Consensus Rate

**Important**:
- Orchestrator uses `agent.chat()` to avoid double-invoke bug that caused token explosion
- SDK retries are enabled (default `max_retries=2`) for graceful rate limit handling
- Application-level retry logic ensures rate limit wait times are **excluded** from timing metrics
- Failed debates are **excluded** from efficiency calculations
- Token metrics are not tracked to maintain accuracy (avoiding estimation inaccuracies)

**Note**: For backtesting performance metrics (Sharpe ratio, returns), use the separate `backtesting.ipynb` notebook with the full 30-stock KLCI benchmark.

## 1. Setup and Imports

In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
from typing import List, Dict
import time

# Import RoundtableAI components
from agents.orchestrator import DebateOrchestrator
from agents.debate_metrics import ModelComparisonMetrics
from utils.database import get_mongo_collection

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

print("✓ Imports successful")

✓ Imports successful


## 2. Configuration

In [2]:
# Models to compare
MODELS_TO_TEST = [
    "gemini-2.0-flash",       # Balanced (baseline)
    "gemini-2.5-pro",         # High quality, slower
    "gemini-2.0-flash-lite",  # Optimized for speed
]

# Debate parameters (same for all models for fair comparison)
MAX_ROUNDS = 5
CONSENSUS_THRESHOLD = 0.75
RISK_TOLERANCE = "moderate"

# Test configuration
NUM_TEST_STOCKS = 5  # Sufficient for efficiency comparison

# Results directory
RESULTS_DIR = "results"
os.makedirs(RESULTS_DIR, exist_ok=True)

print(f"Testing {len(MODELS_TO_TEST)} models on {NUM_TEST_STOCKS} stocks")
print(f"Models: {', '.join(MODELS_TO_TEST)}")

Testing 3 models on 5 stocks
Models: gemini-2.0-flash, gemini-2.5-pro, gemini-2.0-flash-lite


## 3. Load KLCI Test Stocks

In [3]:
# Load KLCI constituents
klci_stocks_df = pd.read_csv("test_dataset.csv")

# Select subset for testing
test_stocks = klci_stocks_df.head(NUM_TEST_STOCKS)

print(f"\nTest stocks ({len(test_stocks)}):")
print(test_stocks[['ticker', 'company_name']].to_string(index=False))

# Prepare stock list
stock_list = test_stocks.to_dict('records')


Test stocks (5):
 ticker       company_name
1155.KL            Maybank
1023.KL               CIMB
5347.KL    Tenaga Nasional
1295.KL        Public Bank
5183.KL Petronas Chemicals


## 4. Helper Functions

In [4]:
def run_debate_with_metrics(orchestrator, company_name, ticker, max_retries=3):
    """
    Run a single debate and return result with metrics.
    
    Implements retry logic for rate limits without counting wait time in metrics.
    
    Args:
        orchestrator: DebateOrchestrator instance
        company_name: Company name
        ticker: Stock ticker
        max_retries: Maximum number of retries for rate limit errors
    
    Returns:
        Tuple of (result, metrics, success, error, rate_limit_hits)
    """
    import re
    
    rate_limit_hits = 0
    last_error = None
    
    for attempt in range(max_retries):
        try:
            # Run debate - timing is tracked internally
            result = orchestrator.run_debate(company_name, risk_tolerance=RISK_TOLERANCE)
            metrics = result.metrics if hasattr(result, 'metrics') else None
            
            # Return with rate limit hit count
            return result, metrics, True, None, rate_limit_hits
            
        except Exception as e:
            error_str = str(e)
            last_error = error_str
            
            # Check if this is a rate limit error
            is_rate_limit = any(indicator in error_str.lower() for indicator in [
                'rate limit',
                'quota',
                '429',
                'resource exhausted',
                'too many requests'
            ])
            
            if is_rate_limit and attempt < max_retries:
                rate_limit_hits += 1
                
                # Extract suggested wait time from error message
                wait_time = 60  # Default wait time
                
                backoff_multiplier = 2 ** attempt
                total_wait = min(wait_time * backoff_multiplier, 300)  # Cap at 5 minutes
                
                print(f"  ⚠️  Rate limit hit (attempt {attempt + 1}/{max_retries})")
                print(f"  ⏳ Waiting {total_wait}s before retry (not counted in metrics)...")
                
                # Wait WITHOUT this time being counted in metrics
                # (because we'll create a fresh orchestrator/debate session)
                time.sleep(total_wait)
                
                # Continue to next retry attempt
                continue
            else:
                # Non-retryable error or max retries exceeded
                return None, None, False, last_error, rate_limit_hits
    
    # Should never reach here, but just in case
    return None, None, False, last_error, rate_limit_hits

print("✓ Helper functions defined")

✓ Helper functions defined


## 5. Run Model Comparison

In [ ]:
# Store all results
all_model_results = {}
all_model_metrics = {}

print("="*80)
print("STARTING MODEL COMPARISON EXPERIMENT")
print("="*80)
print(f"Test Stocks: {len(stock_list)}")
print(f"Models: {len(MODELS_TO_TEST)}")
print("="*80)

for model_name in MODELS_TO_TEST:
    print(f"\n{'='*80}")
    print(f"MODEL: {model_name}")
    print(f"{'='*80}")
    
    # Initialize orchestrator for this model
    print(f"Initializing orchestrator...")
    try:
        orchestrator = DebateOrchestrator(
            model_name=model_name,
            max_rounds=MAX_ROUNDS,
            consensus_threshold=CONSENSUS_THRESHOLD,
            track_metrics=True
        )
    except Exception as e:
        print(f"❌ Failed to initialize {model_name}: {e}")
        print(f"Skipping {model_name}...\n")
        continue
    
    # Store results for this model
    model_results = []
    model_comparison = ModelComparisonMetrics(model_name=model_name)
    
    # Track rate limit statistics
    total_rate_limit_hits = 0
    
    # Run debate for each stock
    for idx, stock in enumerate(stock_list, 1):
        ticker = stock['ticker']
        company = stock['company_name']
        
        print(f"\n[{idx}/{len(stock_list)}] {company} ({ticker})")
        
        # Run debate with retry logic
        result, metrics, success, error, rate_limit_hits = run_debate_with_metrics(
            orchestrator, company, ticker
        )
        
        total_rate_limit_hits += rate_limit_hits
        
        if not success:
            print(f"  ❌ Error: {error}")
            model_results.append({
                'ticker': ticker,
                'company': company,
                'success': False,
                'error': error,
                'rate_limit_hits': rate_limit_hits
            })
            # IMPORTANT: Do NOT add failed debates to metrics
            continue
        
        # Store result
        recommendation = result.recommendation.value
        confidence = result.confidence
        
        print(f"  Recommendation: {recommendation} (confidence: {confidence:.0%})")
        
        if metrics:
            print(f"  Time: {metrics.total_time:.1f}s | Rounds: {metrics.rounds_completed}")
            if rate_limit_hits > 0:
                print(f"  ⚠️  Rate limit hits: {rate_limit_hits} (wait time excluded from metrics)")
            
            # Only add successful debates to efficiency metrics
            model_comparison.add_debate(metrics)
        
        model_results.append({
            'ticker': ticker,
            'company': company,
            'recommendation': recommendation,
            'confidence': confidence,
            'success': True,
            'rate_limit_hits': rate_limit_hits,
            'metrics': metrics.to_dict() if metrics else None
        })
        
        # Small delay between stocks to avoid rate limits
        time.sleep(1)
    
    # Store model results
    all_model_results[model_name] = model_results
    all_model_metrics[model_name] = model_comparison
    
    # Print model summary
    print(f"\n{'-'*80}")
    print(f"SUMMARY: {model_name}")
    print(f"{'-'*80}")
    successful = sum(1 for r in model_results if r['success'])
    failed = sum(1 for r in model_results if not r['success'])
    print(f"Completed: {successful}/{len(stock_list)} stocks")
    if failed > 0:
        print(f"Failed: {failed} (excluded from efficiency metrics)")
    if total_rate_limit_hits > 0:
        print(f"Rate limit hits: {total_rate_limit_hits} (wait time excluded from metrics)")
    model_comparison.print_summary()

print(f"\n{'='*80}")
print("EXPERIMENT COMPLETE")
print(f"{'='*80}")

STARTING MODEL COMPARISON EXPERIMENT
Test Stocks: 5
Models: 3

MODEL: gemini-2.0-flash
Initializing orchestrator...
Initializing Gemini model: gemini-2.0-flash
Successfully connected to Gemini model: gemini-2.0-flash

[1/5] Maybank (1155.KL)


In [ ]:
# Efficiency metrics comparison (only includes successful debates)
efficiency_comparison = []

for model_name, comparison in all_model_metrics.items():
    # Get count of successful vs failed debates
    results = all_model_results[model_name]
    successful_count = sum(1 for r in results if r['success'])
    failed_count = sum(1 for r in results if not r['success'])
    rate_limit_hits = sum(r.get('rate_limit_hits', 0) for r in results)
    
    efficiency_comparison.append({
        'Model': model_name,
        'Successful': successful_count,
        'Failed': failed_count,
        'Rate Limit Hits': rate_limit_hits,
        'Avg Total Time (s)': comparison.avg_total_time,
        'Time/Round (s)': comparison.avg_time_per_round,
        'Time/Agent (s)': comparison.avg_time_per_agent,
        'Avg Rounds': comparison.avg_rounds,
        'Consensus Rate': comparison.consensus_rate
    })

efficiency_df = pd.DataFrame(efficiency_comparison)

print("\n" + "="*80)
print("EFFICIENCY COMPARISON")
print("="*80)
print("\nNote: Metrics calculated from successful debates only.")
print("Rate limit wait times are excluded from timing metrics.\n")
print(efficiency_df.to_string(index=False))

## 6. Create Efficiency Comparison Table

## 7. Save Results

In [ ]:
# Prepare export data
export_data = {
    'experiment_date': datetime.now().isoformat(),
    'configuration': {
        'models': MODELS_TO_TEST,
        'num_stocks': NUM_TEST_STOCKS,
        'max_rounds': MAX_ROUNDS,
        'consensus_threshold': CONSENSUS_THRESHOLD,
        'risk_tolerance': RISK_TOLERANCE
    },
    'notes': {
        'rate_limit_handling': 'Rate limit wait times are excluded from timing metrics',
        'failed_debates': 'Failed debates are excluded from efficiency calculations',
        'retry_logic': 'Up to 3 retries with exponential backoff for rate limit errors'
    },
    'efficiency_metrics': {
        model: comp.to_dict()
        for model, comp in all_model_metrics.items()
    },
    'detailed_results': all_model_results
}

# Save as JSON
json_path = os.path.join(RESULTS_DIR, 'efficiency_comparison.json')
with open(json_path, 'w') as f:
    json.dump(export_data, f, indent=2)
print(f"\n✓ Saved: {json_path}")

# Save efficiency table as CSV
csv_path = os.path.join(RESULTS_DIR, 'efficiency_comparison.csv')
efficiency_df.to_csv(csv_path, index=False)
print(f"✓ Saved: {csv_path}")

print("\n" + "="*80)
print("RESULTS SAVED SUCCESSFULLY")
print("="*80)

## 8. Visualization

In [ ]:
print("\n" + "="*80)
print("EXPERIMENT SUMMARY")
print("="*80)

print(f"\nModels Tested: {len(MODELS_TO_TEST)}")
print(f"Stocks Analyzed: {NUM_TEST_STOCKS}")
print(f"Total Debates: {sum(len(r) for r in all_model_results.values())}")

# Calculate success/failure statistics
total_successful = sum(sum(1 for r in results if r['success']) for results in all_model_results.values())
total_failed = sum(sum(1 for r in results if not r['success']) for results in all_model_results.values())
total_rate_limits = sum(sum(r.get('rate_limit_hits', 0) for r in results) for results in all_model_results.values())

print(f"Successful: {total_successful}")
if total_failed > 0:
    print(f"Failed: {total_failed} (excluded from efficiency metrics)")
if total_rate_limits > 0:
    print(f"Total Rate Limit Hits: {total_rate_limits} (wait time excluded from metrics)")

print("\n" + "-"*80)
print("KEY FINDINGS (Based on Successful Debates Only)")
print("-"*80)

# Filter to models with at least one successful debate
valid_df = efficiency_df[efficiency_df['Successful'] > 0].copy()

if len(valid_df) == 0:
    print("\n⚠️  No successful debates to analyze")
else:
    # Fastest model (lowest time/round)
    fastest = valid_df.loc[valid_df['Time/Round (s)'].idxmin()]
    print(f"\n⚡ FASTEST: {fastest['Model']}")
    print(f"   Time/Round: {fastest['Time/Round (s)']:.2f}s")
    print(f"   Success Rate: {fastest['Successful']}/{fastest['Successful'] + fastest['Failed']}")

    # Most efficient convergence (fewest rounds)
    best_convergence = valid_df.loc[valid_df['Avg Rounds'].idxmin()]
    print(f"\n🎯 BEST CONVERGENCE: {best_convergence['Model']}")
    print(f"   Avg Rounds: {best_convergence['Avg Rounds']:.1f}")
    print(f"   Total Time: {best_convergence['Avg Total Time (s)']:.1f}s")

    # Overall fastest (total time)
    overall_fastest = valid_df.loc[valid_df['Avg Total Time (s)'].idxmin()]
    print(f"\n⚖️ OVERALL FASTEST: {overall_fastest['Model']}")
    print(f"   Total Time: {overall_fastest['Avg Total Time (s)']:.1f}s")
    print(f"   Consensus Rate: {overall_fastest['Consensus Rate']:.0%}")

print("\n" + "="*80)
print("Files saved to:", RESULTS_DIR)
print("  - efficiency_comparison.json")
print("  - efficiency_comparison.csv")
print("  - efficiency_visualization.png")
print("="*80)

print("\nℹ️  IMPORTANT NOTES:")
print("  • Rate limit wait times are EXCLUDED from all timing metrics")
print("  • Failed debates are EXCLUDED from efficiency calculations")
print("  • Only successful debates contribute to performance statistics")
print("\n  For backtesting performance metrics (Sharpe ratio, returns),")
print("  run the backtesting.ipynb notebook with the full 30-stock KLCI benchmark.")

## 9. Summary Report

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

# Filter to models with successful debates
plot_df = efficiency_df[efficiency_df['Successful'] > 0].copy()

if len(plot_df) == 0:
    print("⚠️  No successful debates to visualize")
else:
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    models = plot_df['Model'].tolist()
    colors = ['#4CAF50', '#2196F3', '#FF9800'][:len(models)]
    
    # 1. Time/Round
    ax = axes[0, 0]
    ax.bar(range(len(models)), plot_df['Time/Round (s)'], color=colors)
    ax.set_xticks(range(len(models)))
    ax.set_xticklabels([m.replace('gemini-', '') for m in models], rotation=45, ha='right')
    ax.set_ylabel('Seconds')
    ax.set_title('Time per Round (Lower is Better)')
    ax.grid(axis='y', alpha=0.3)
    
    # 2. Time/Agent
    ax = axes[0, 1]
    ax.bar(range(len(models)), plot_df['Time/Agent (s)'], color=colors)
    ax.set_xticks(range(len(models)))
    ax.set_xticklabels([m.replace('gemini-', '') for m in models], rotation=45, ha='right')
    ax.set_ylabel('Seconds')
    ax.set_title('Time per Agent Response (Lower is Better)')
    ax.grid(axis='y', alpha=0.3)
    
    # 3. Consensus Rate
    ax = axes[0, 2]
    consensus_pct = plot_df['Consensus Rate'] * 100
    ax.bar(range(len(models)), consensus_pct, color=colors)
    ax.set_xticks(range(len(models)))
    ax.set_xticklabels([m.replace('gemini-', '') for m in models], rotation=45, ha='right')
    ax.set_ylabel('Percentage (%)')
    ax.set_title('Consensus Rate (Higher is Better)')
    ax.set_ylim([0, 110])
    ax.grid(axis='y', alpha=0.3)
    
    # 4. Total Time
    ax = axes[1, 0]
    ax.bar(range(len(models)), plot_df['Avg Total Time (s)'], color=colors)
    ax.set_xticks(range(len(models)))
    ax.set_xticklabels([m.replace('gemini-', '') for m in models], rotation=45, ha='right')
    ax.set_ylabel('Seconds')
    ax.set_title('Avg Total Debate Time (Lower is Better)')
    ax.grid(axis='y', alpha=0.3)
    
    # 5. Avg Rounds
    ax = axes[1, 1]
    ax.bar(range(len(models)), plot_df['Avg Rounds'], color=colors)
    ax.set_xticks(range(len(models)))
    ax.set_xticklabels([m.replace('gemini-', '') for m in models], rotation=45, ha='right')
    ax.set_ylabel('Rounds')
    ax.set_title('Avg Rounds to Completion')
    ax.grid(axis='y', alpha=0.3)
    
    # 6. Success Rate & Rate Limit Hits
    ax = axes[1, 2]
    
    # Calculate success rate percentage
    success_rates = []
    for _, row in plot_df.iterrows():
        total = row['Successful'] + row['Failed']
        success_rate = (row['Successful'] / total * 100) if total > 0 else 0
        success_rates.append(success_rate)
    
    x = range(len(models))
    width = 0.35
    
    # Bar for success rate
    bars1 = ax.bar([i - width/2 for i in x], success_rates, width, 
                   label='Success Rate (%)', color=colors, alpha=0.7)
    
    # Bar for rate limit hits
    bars2 = ax.bar([i + width/2 for i in x], plot_df['Rate Limit Hits'], width,
                   label='Rate Limit Hits', color='red', alpha=0.5)
    
    ax.set_xticks(x)
    ax.set_xticklabels([m.replace('gemini-', '') for m in models], rotation=45, ha='right')
    ax.set_ylabel('Count / Percentage')
    ax.set_title('Success Rate & Rate Limit Hits')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    plt.suptitle('Model Comparison: Efficiency Metrics\n(Rate limit wait times excluded)', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    # Save
    viz_path = os.path.join(RESULTS_DIR, 'efficiency_visualization.png')
    plt.savefig(viz_path, dpi=150, bbox_inches='tight')
    print(f"\n✓ Saved visualization: {viz_path}")
    
    plt.show()